# Introduction

This tutorial will introduce you to the basics of the decision tree model, how to implement a classification decision tree, and how decision trees are used in real world applications, specifically in health and diagnosis of illnesses. Given some specific details about a person, can we predict if they have some illness, for example cancer or the flu? Many illnesses will have a list of symptoms where if you were to be experiencing some of them, you could have a high chance of having that illness. And so, decision trees are a great tool that will not only help make these predictions based on certain details, but also they provide a great visual that is very easy to understand.

# Decision Trees

A decision tree is a tree diagram where at each node, a decision can be made such that you follow the branches corresponding to which decision you made, and ultimately ending up at a leaf node with some possible outcome to the problem. At each internal node of a decision tree, there will be one value attribute, $x_i$, which can either be a discrete value or a continuous value. Each branch from a node selects one value for $x_i$, and each leaf node predicts the outcome $Y$. There are two main types of decision trees: classification trees and regression trees. In this tutorial, we will be working with classification decision trees which means that our outcome prediction is a discrete value. The regression tree predicts outcomes that can be considered real numbers.

# Tutorial Content

In this tutorial, we will be focusing on implementing a classficiation decision tree in Python to decide given a person's symptoms, whether they have COVID or not. In this dataset, we will be working with discrete attributes as well as a discrete outcome. We will be using a dataset from Kaggle that includes symptoms as the attributes and COVID presence as the outcome: https://www.kaggle.com/hemanthhari/symptoms-and-covid-presence

We will cover the following topics in this tutorial:
- [Libraries](#libraries)
- [Data Structure for Decision Tree](#datastructure)
- [General Algorithm for Decision Tree](#general)
- [Loading the Data](#loading)
- [Entropy and Information Gain Calculations](#entropy)
- [Building the Decision Tree](#building)
- [Making Predictions](#predictions)
- [Error Rate and Overfitting](#error)
- [Summary and References](#summary)

<a id='libraries'></a>
# Libraries

We will need the following libraries in order to implement our decision tree. Run the following code:

In [1]:
import numpy as np
import pandas as pd

<a id='datastructure'></a>
# Data Structure for Decision Tree

Because we are implementing a decision tree, it makes the most sense to represent our decision tree using an actual tree data structure. We will define a node class with different attributes that will help facilitate the implementation for our decision tree and then build a tree with these node objects.
- self.left: this will be the left child of the node
- self.right: this will be the right child of the node
- self.data: this contains the current dataset we are working with
- self.attribute: the attribute that is split on at the current node
- self.colIndex: this is the column index of the attribute in our data table
- self.maxDepth: how deep we want our tree

In [2]:
class Node:
    def __init__(self, data, depth):
        self.left = None
        self.right = None
        self.data = data
        self.attribute = None
        self.colIndex = None
        self.maxDepth = depth

<a id='general'></a>
# General Algorithm for Decision Tree

Here is a general outline for how to build a decision tree:

1. Start with an empty tree at the root, which is just an empty node and your training dataset
2. Do the following until some stopping criterion is met:
       a) pick the best attribute to put at the node, let this attribute be called A
       b) assign A as the decision attribute for the current empty node you are at
       c) for each value of A, create a new descendent of node (the branches)
       d) sort training examples to leaf nodes
       e) repeat loop at leaf nodes
3. Output decision tree

This tutorial will walk us through the above steps.

<a id='loading'></a>
# Loading the Data

The first thing we need to do is download and load our data into some form that we can use easily when implementing our decision tree. You are going to click Download from this Kaggle webpage: https://www.kaggle.com/hemanthhari/symptoms-and-covid-presence (note: you will need an account in order to download datasets from Kaggle). Then, unzip the archieve.zip file to create an archive folder which will include a file named Covid dataset in the form of a CSV file. So then in order for us to use this dataset, we need the Covid dataset file to be in the the same folder as this notebook, and then we can load the data by using the following commands: 

In [3]:
dataframe = pd.read_csv("Covid_Dataset.csv", delimiter=",", quotechar='"')
dataframe.head()

,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,Yes,Yes,Yes,Yes,Yes,No,No,No,No,Yes,...,Yes,Yes,No,Yes,No,Yes,Yes,No,No,Yes
1,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,No,No,...,Yes,No,No,No,Yes,Yes,No,No,No,Yes
2,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,...,Yes,Yes,Yes,No,No,No,No,No,No,Yes
3,Yes,Yes,Yes,No,No,Yes,No,No,Yes,Yes,...,No,No,Yes,No,Yes,Yes,No,No,No,Yes
4,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,...,No,Yes,No,Yes,No,Yes,No,No,No,Yes


<a id='entropy'></a>
# Entropy and Information Gain 

Before we can move any further with our implementation of the decision tree, we need to decide how to choose the "best" attribute at each node. For this, we need something called entropy and information gain. Entropy measures the impurity of a collection of examples (how separable the data are). For a decision tree, we want a lower entropy so that our data is more pure. <br>
The entropy H(Y) for a random variable Y with n possible values is: <br>
$H(Y) = - \sum \limits _{i=1} ^{n} P(Y = i)log_2P(Y=i)$ <br>
So then to decide which attribute decreases the most entropy for the dataset we use information gain, also called mutual information. Information gain, $I_S(Y, A)$ is the reduction in entropy of target variable Y for data sample S, due to sorting on a variable A. So when looking at different attributes, we want the attribute that gives us the highest information gain, resulting in the largest decrease to the entropy of our whole dataset. In order to calculate information gain, we will need additional formulas. <br>
Specific conditional entropy H(Y|X = v) of Y given X = v: <br>
$H(Y|X = v) = 0 \sum \limits _{i=1} ^{n} P(Y = i|X = v)log_2P(Y = i|X = v)$ <br>
Conditional entropy H(Y|X) of Y given X: <br>
$H(Y|X) = \sum \limits _{v \in values(X)} ^{} P(X = v)H(Y|X = v)$ <br>
So then information gain, $I(Y, X)$, is calculated by: <br>
$I(Y, X) = H(Y) - H(Y|X)$

Now that we have the formulas needed to find the best attribute at some node, let's implement them into code! First, let's implement a function that just calculates the total entropy of the dataset, H(Y).

In [4]:
def totalEntropyCalc(root):
    y_yes = root.data.loc[root.data['COVID-19'] == 'Yes']
    y_no = root.data.loc[root.data['COVID-19'] == 'No']
    count1 = len(y_yes)
    count2 = len(y_no)
    total = count1+count2
    if (count1==0 and count2 == 0): return 0
    if(count1 ==0):
        entropy = (count2/float(total))*np.log2(count2/float(total))
        return (-1*entropy)
    if(count2==0):
        entropy = (count1/float(total))*np.log2(count1/float(total))
        return (-1*entropy)
    entropy = (count1/float(total))*np.log2(count1/float(total)) + (count2/float(total))*np.log2(count2/float(total))
    return (-1*entropy)

Next, we will need a function that partitions our data based on some attribute A so we can determine the proportion of people with COVID given that they either have attribute A or don't have attribute A and the proportion of people who don't have COVID given that they either have attribute A or don't have attribute A. These numbers will be needed to calculate the specific conditional entropy.

In [5]:
def partition(colAttribute, root):
    x_yes = root.data.loc[root.data[colAttribute] == 'Yes']
    x_no = root.data.loc[root.data[colAttribute] == 'No']
    xYes_yYes = x_yes.loc[x_yes['COVID-19'] == 'Yes']
    xYes_yNo = x_yes.loc[x_yes['COVID-19'] == 'No']
    xNo_yYes = x_no.loc[x_no['COVID-19'] == 'Yes']
    xNo_yNo = x_no.loc[x_no['COVID-19'] == 'No']
    return (x_yes, xYes_yYes, xYes_yNo, x_no, xNo_yYes, xNo_yNo)

Now that we have partitioned our data given some attribute A, we can now calculate the conditional entropy. 

In [6]:
def entropyCalc(y_yes, y_no):
    count1 = len(y_yes)
    count2 = len(y_no)
    if (count1 == 0 and count2 == 0): return 0
    total = count1+count2
    if (count1 == 0):
        entropy = entropy = (count2/float(total))*np.log2(count2/float(total))
        return (-1*entropy)
    if(count2 == 0):
        entropy = (count1/float(total))*np.log2(count1/float(total))
        return (-1*entropy)
    entropy = (count1/float(total))*np.log2(count1/float(total)) + (count2/float(total))*np.log2(count2/float(total))
    return (-1*entropy)

And now finally, we can implement a function to calculate the information gain for some attribute.

In [7]:
def mutualInfoCalc(totalEntropy, entropy1, x_yes, entropy2, x_no):
    count1 = len(x_yes)
    count2 = len(x_no)
    total = count1+count2
    infoGain = totalEntropy - ( (count1/float(total))*entropy1 + (count2/float(total))*entropy2 )
    return infoGain

Now that we have the way to find the best attribute to split at for each node, we will implement another function to help with printing out our decision tree at the end. It is very useful to see the actual proportions for the data at each node, so we will keep count of the majority portion and the minority portion of data at every node.

In [8]:
def majorityVote(root):
    y_yes = root.data.loc[root.data['COVID-19'] == 'Yes']
    y_no = root.data.loc[root.data['COVID-19'] == 'No']
    count_yes = len(y_yes)
    count_no = len(y_no)
    
    if (count_yes > count_no):
        majority = 'Yes'
        minority = 'No'
        return(majority, count_yes, minority, count_no)
    else:
        majority = 'No'
        minority = 'Yes'
        return(majority, count_no, minority, count_yes)
        

The last function before we get to building our tree is to help split the data based on a single attribute. So once we find the best attribute, we need to split our data based on the whether a person is 'yes' for attribute A or 'no'. So essentially we are going to use this function to build a decision stump (which is a decision tree with depth 1) at every node, just so we can separate the building of our full decision tree into smaller, and more clear functions.

In [9]:
def decisionStump(colAttribute, root):
    x_yes = root.data.loc[root.data[colAttribute] == 'Yes']
    x_no = root.data.loc[root.data[colAttribute] == 'No']
    count_yes = len(x_yes)
    count_no = len(x_no)
    if(count_yes > count_no):
        return(x_yes, x_no)
    else:
        return(x_no, x_yes)

<a id='building'></a>
# Building the Decision Tree

So we now have all of our functions with attribute choosing and printing the tree, let's start building the tree! First, we must initialize our tree to be an empty node, containing the entire dataset. Here, we are initializing the max depth of our tree to be 1 just to demonstrate splitting on one attribute, but later we will look at the affects of changing max depth.

In [10]:
root = Node(dataframe, 1)

Now, let's implement the main loop for the decision tree algorithm. We will want to calculate the information gain of every attribute, find the largest one, and then split our data accordingly. We continue this process until some stopping criteria is met. There are three base cases. One stopping criteria is when we have reached our max depth. The max depth is a hyperparameter, which means that it is not derived through the learning process, but rather chosen outside of the model and can help make the model a better fit for the data and a better predictor. Another stopping criteria is when the total entropy of the dataset is zero. This means that our data is perfectly classified and so there is no need to continue splitting. The last base case to consider is when all the information gains are zero. This indicates that we have already used all the attributes to split on and there are no more attributes to consider.

In [11]:
def treeBuilder(root, depth, attributes, printBool):
    #Base Case: Max depth reached
    if (depth >= root.maxDepth):
        return (root)
    totalEntropy = totalEntropyCalc(root)
    #Base Case: Data is perfectly classified
    if (totalEntropy == 0):
        return (root)
    infoGains = []
    for attribute in attributes:
        (x_yes, xYes_yYes, xYes_yNo, x_no, xNo_yYes, xNo_yNo) = partition(attribute, root)
        entropy1 = entropyCalc(xYes_yYes, xYes_yNo)
        entropy2 = entropyCalc(xNo_yYes, xNo_yNo)
        infoGain = mutualInfoCalc(totalEntropy, entropy1, x_yes, entropy2, x_no)
        infoGains = infoGains + [infoGain]
    #Base Case: all attributes have been used
    if (sum(infoGains) == 0):
        return (root)
    #Recursive Case:
    else:
        if (depth == 0):
            (major, majorVote, minor, minorVote) = majorityVote(root)
            if printBool:
                print("[" + str(majorVote) + " " + major + "/" + str(minorVote) + " " +  minor + "]")
        maxIG = -1
        splitCol = -1
        #find the attribute that gives the highest information gain
        for i in range(len(infoGains)):
            if infoGains[i] > maxIG:
                splitCol = i
                maxIG = infoGains[i]
        #split on best attribute
        root.attribute = attributes[splitCol]
        root.splitCol = splitCol
        root.colIndex = splitCol
        (leftData, rightData) = decisionStump(attributes[splitCol], root)
        if (len(leftData) < len(rightData)):
            root.right = Node(leftData, root.maxDepth)
            root.left = Node(rightData, root.maxDepth)
        else:
            root.left = Node(leftData, root.maxDepth)
            root.right = Node(rightData, root.maxDepth)
        (mL, mVotesL, minL, minVotesL) = majorityVote(root.left)
        if printBool:
            print(" | " * (depth+1), root.attribute + " = " + root.left.data.iloc[0][splitCol] + ": [" + str(mVotesL) + " " + mL + "/" + str(minVotesL) + " " + minL + "]")
        treeBuilder(root.left, depth+1, attributes, printBool)
        (mR, mVotesR, minR, minVotesR) = majorityVote(root.right)
        if printBool:
            print(" | " * (depth+1), root.attribute + " = " + root.right.data.iloc[0][splitCol] + ": [" + str(mVotesR) + " " + mR + "/" + str(minVotesR) + " " + minR + "]")
        treeBuilder(root.right, depth+1, attributes, printBool)

Now that we have our tree builder function, let's build our first decision tree with a depth of 1.

In [12]:
colAttributes = list(dataframe.columns)
colAttributes = colAttributes[0 : (len(colAttributes)-1)]
decision_tree = treeBuilder(root, 0, colAttributes, True)

[4383 Yes/1051 No]
 |  Abroad travel = No: [1932 Yes/1051 No]
 |  Abroad travel = Yes: [2451 Yes/0 No]


Looking at our decision tree, we can see that the best attribute to split on is 'Abroad travel' because it leads to the most pure dataset. Now let's see what our tree looks like with a larger max depth.

In [13]:
root = Node(dataframe, 10)
decision_tree = treeBuilder(root, 0, colAttributes, True)

[4383 Yes/1051 No]
 |  Abroad travel = No: [1932 Yes/1051 No]
 |  |  Sore throat = Yes: [1638 Yes/284 No]
 |  |  |  Attended Large Gathering = Yes: [977 Yes/0 No]
 |  |  |  Attended Large Gathering = No: [661 Yes/284 No]
 |  |  |  |  Contact with COVID Patient = No: [277 Yes/252 No]
 |  |  |  |  |  Breathing Problem = Yes: [232 Yes/92 No]
 |  |  |  |  |  |  Fever = Yes: [212 Yes/49 No]
 |  |  |  |  |  |  |  Family working in Public Exposed Places = No: [112 Yes/49 No]
 |  |  |  |  |  |  |  |  Asthma = Yes: [54 Yes/38 No]
 |  |  |  |  |  |  |  |  |  Headache = No: [38 No/33 Yes]
 |  |  |  |  |  |  |  |  |  |  Fatigue  = Yes: [38 No/28 Yes]
 |  |  |  |  |  |  |  |  |  |  Fatigue  = No: [5 Yes/0 No]
 |  |  |  |  |  |  |  |  |  Headache = Yes: [21 Yes/0 No]
 |  |  |  |  |  |  |  |  Asthma = No: [58 Yes/11 No]
 |  |  |  |  |  |  |  |  |  Heart Disease = No: [37 Yes/0 No]
 |  |  |  |  |  |  |  |  |  Heart Disease = Yes: [21 Yes/11 No]
 |  |  |  |  |  |  |  |  |  |  Dry Cough = Yes: [16 Yes/1

As you can see, now we have a decision tree of depth 10. This decision tree can be easily followed because the lines on the left side tell us the depth of the node we are looking at. Branches are indicated by indents and the number of lines tell us how deep we are into the tree. When you reach a line that has no more indents, you have reached a leaf node because one of the stopping criteria has been met. For example, on the fourth line down from the top, Attended Large Gathering = Yes: [977 Yes/0 No], although this does not have depth 10, it no longer indents and splits anymore because this data is perfectly classified since all the examples are 'Yes'. At the line that says, Fatigue  = Yes: [38 No/28 Yes], this no longer splits further because the max depth of 10 was reached. When reading this tree top to bottom and you reach a leaf node, we explore the opposite path based on the previous node. At each line, you have the attribute name and then after the equals sign, you will then have a 'Yes' or 'No' value. This indicates whether a person has the attribute or does not have the attribute. Then, we see a ratio of 'Yes' and 'No' responses in brackets. These numbers indicate the subset of people who have COVID, based on their value for that attribute. In order to make a prediction based on someone's attribute, you take the larger proportion of people from the brackets. So for example, if we had someone who traveled abroad, we would predict that they do have COVID because following the tree, we see that Abroad travel = Yes: [2451 Yes/0 No] and the majority of people who have traveled abroad have COVID and so we predict that future people will also have COVID if they travel abroad.

<a id='predictions'></a>
# Making Predictions

Now what can we do with a decision tree? We can make predictions! So we will need to follow the branches on the tree based on our sample example and then come with up a prediction whether they have COVID or not. Let's implement the predict function.

In [14]:
def predictions(root, data):
    pred = []
    dummyRoot = root
    for row in data:
        for i in range(root.maxDepth):
            if root.colIndex == None:
                break
            x = row[root.colIndex]
            if x == root.left.data.iloc[0][root.colIndex]:
                root = root.left
            else:
                root = root.right
        (maj, majC, mino, minC) = majorityVote(root)
        guess = maj
        root = dummyRoot
        pred += [guess]
    return pred

Let's see what we predict on our training data, when we have a decision tree of depth 5.

In [15]:
root = Node(dataframe, 5)
decision_tree = treeBuilder(root, 0, colAttributes, False)
trainData = dataframe.values.tolist()
trainPreds = predictions(root, trainData)
print(trainPreds[0:10])

['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']


Based on our output, we predict that the first 10 samples from our dataset all have COVID, following the decision tree with depth 5.

<a id='error'></a>
# Error Rate and Overfitting

After making our predictions, it would be useful to see how well our model performed. We will look at the error rate and see what percentage of examples our model predicted correctly.

In [16]:
def errorRate(predictions, real):
    total = 0
    wrong = 0
    for index in range(len(predictions)):
        if predictions[index] != real[index]:
            wrong += 1
        total += 1
    return (wrong / float(total))

In [17]:
realLabels = dataframe['COVID-19']
realLabels = realLabels.values.tolist()
errorRate(trainPreds, realLabels)

0.04766286345233714

So we see that when we train our decision tree to have a depth of 5, our error rate is about 4.76%! Let's play around and see what happens with the error rate as we change the depth of the tree.

In [18]:
for i in range(0, len(colAttributes), 4):
    root = Node(dataframe, i)
    decision_tree = treeBuilder(root, 0, colAttributes, False)
    trainData = dataframe.values.tolist()
    trainPreds = predictions(root, trainData)
    print('Depth = ' + str(i) + " : error rate = ", errorRate(trainPreds, realLabels))

Depth = 0 : error rate =  0.19341185130658814
Depth = 4 : error rate =  0.0826278984173721
Depth = 8 : error rate =  0.024475524475524476
Depth = 12 : error rate =  0.01729849098270151
Depth = 16 : error rate =  0.01729849098270151


As we can see, as the depth of our tree increases, the error rate decreases, telling us that our model is getting better on the training data. This should make sense because as we split on more attributes, we are getting more and more specific and we can correctly classify the training examples. However, this does not mean we should always make our decision tree have a max depth where we use all possible attributes. This will cause overfitting because although our decision tree is getting better at predicting our training examples, we have no idea how well it will do on unseen data. Our model could very well be fitting the training data, but the training data is not always an accurate picture of unseen data and so we don't want our model to be too specific for just our training data. 

One way to avoid overfitting your decision tree is called reduced error pruning. What you want to do is split your training data into a new training dataset as well as a validation set. It is typical to split 80%/20%. Then, using your new training set, train your tree so that it classifies the training dataset as best as possible (produces lowest possible error rate). Then using the validation set, we are going to prune the tree so that we produce the lowest error rate on the validation set. To do this, at each internal node, we consider turning it into a leaf node and pruning the tree below it. Then using this pruned tree, make predictions with the validation set and get the error rate. Keep repeating this process with every internal node, then choose the tree that gives the lowest validation error rate.

Once you have chosen the best depth for your tree, you can go ahead and run your model on some new unseen data (test data) and see how accurately your model makes predictions. However, after running your tree on test data, you should not go back and revise your model based on the test data. Doing so would harm the integrity of your original model.

<a id='summary'></a>
# Summary and References

This tutorial outlined the general algorithm for building a classification decision tree in Python and using a real world example to highlight the usefulness and benefits to the decision tree model. If you would like to learn more about the decision tree algorithm or look for more datasets to try implementing your own, here are some useful links.

1. COVID dataset: https://www.kaggle.com/hemanthhari/symptoms-and-covid-presence
2. Decision Trees in Machine Learning: https://towardsdatascience.com/decision-trees-in-machine-learning-641b9c4e8052
3. Pandas Library: https://pandas.pydata.org/
4. General Notes about Decision Trees: http://www.cs.cmu.edu/~mgormley/courses/10601/slides/lecture2-dtrees-overfit.pdf